In [135]:
import numpy as np
import gzip
import os
from sklearn import preprocessing
import pickle
from typing import Tuple

In [3]:
class MnistDataLoader:
    def __init__(self, data_folder_path):
        self.data_folder_path = data_folder_path
        self.train_file_name = 'train-images-idx3-ubyte.gz'
        self.train_label_file_name = 'train-labels-idx1-ubyte.gz'
        self.test_file_name = 't10k-images-idx3-ubyte.gz'
        self.test_label_file_name = 't10k-labels-idx1-ubyte.gz'
        self.data = dict()
        self.size = 28
        self.color_channel = 1
        self.data_list = [
            'train_images',
            'train_labels',
            'test_images',
            'test_labels'
        ]

    def load_images(self, data_list_index, file_name):
        images = gzip.open(os.path.join(self.data_folder_path, file_name), 'rb')
        self.data[self.data_list[data_list_index]] = np.frombuffer(images.read(), dtype=np.uint8, offset=16).reshape(-1, self.size, self.size)
        self.data[self.data_list[data_list_index]] = self.data[self.data_list[data_list_index]].reshape(self.data[self.data_list[data_list_index]].shape[0], self.size, self.size, self.color_channel).astype(np.float32)

    def load_labels(self, data_list_index, file_name):
        labels = gzip.open(os.path.join(self.data_folder_path, file_name), 'rb')
        self.data[self.data_list[data_list_index]] = np.frombuffer(labels.read(), dtype=np.uint8, offset=8)
        self.data[self.data_list[data_list_index]].resize(self.data[self.data_list[data_list_index]].shape[0],1)

    def load_mnist(self):
        self.load_images(data_list_index=0, file_name=self.train_file_name)
        self.load_labels(data_list_index=1, file_name=self.train_label_file_name)
        self.load_images(data_list_index=2, file_name=self.test_file_name)
        self.load_labels(data_list_index=3, file_name=self.test_label_file_name)

        self.assert_data_shape()

    def assert_data_shape(self):
        assert self.data[self.data_list[0]].shape == (60000, 28, 28, 1)
        assert self.data[self.data_list[1]].shape == (60000, 1)
        assert self.data[self.data_list[2]].shape == (10000, 28, 28, 1)
        assert self.data[self.data_list[3]].shape == (10000, 1)

    def preprocess_data(self):

        self.data[self.data_list[0]] /= 255
        self.data[self.data_list[2]] /= 255

        self.data[self.data_list[1]] = Utility.one_hot_encode(self.data[self.data_list[1]])

        assert self.data[self.data_list[1]].shape == (60000, 10)

In [48]:
class Cifer10DataLoader:
    def __init__(self, data_path):
        self.data_path = data_path
        self.size = 32
        self.color_channel = 3
        self.per_batch_data_size = 10000
        self.data = dict()

    def load_data(self, file_name):
        with open(os.path.join(self.data_path, file_name), 'rb') as f:

            data_dict=pickle.load(f, encoding='latin1')

            images = data_dict['data']
            labels = data_dict['labels']

            images = images.reshape(self.per_batch_data_size, self.color_channel, self.size, self.size).transpose(0,2,3,1).astype("float")
            labels = np.array(labels)
            print(labels.shape)

            return images, labels

    def concatenate_data(self):
        X1, Y1 = self.load_data('data_batch_1')
        X2, Y2 = self.load_data('data_batch_2')
        X3, Y3 = self.load_data('data_batch_3')
        X4, Y4 = self.load_data('data_batch_4')
        X5, Y5 = self.load_data('data_batch_5')

        self.data['train_images'] = np.concatenate(
            (
                X1, X2, X3, X4, X5
            ),
            axis=0
        )

        self.data['train_labels'] = np.concatenate(
            (
                Y1.reshape(self.per_batch_data_size, 1),
                Y2.reshape(self.per_batch_data_size, 1),
                Y3.reshape(self.per_batch_data_size, 1),
                Y4.reshape(self.per_batch_data_size, 1),
                Y5.reshape(self.per_batch_data_size, 1)
            ),
            axis=0
        )

        X_test, Y_test = self.load_data('test_batch')

        self.data['test_images'] = X_test
        self.data['test_labels'] = Y_test.reshape(Y_test.shape[0], 1)

        self.assert_data_shape()

        for key, data in self.data.items():
            print(f'Shape: {data.shape}')

    def assert_data_shape(self):
        assert self.data['train_images'].shape == (50000, 32, 32, 3)
        assert self.data['train_labels'].shape == (50000, 1)
        assert self.data['test_images'].shape  == (10000, 32, 32, 3)
        assert self.data['test_labels'].shape  == (10000, 1)

    def preprocess_data(self):
        self.data['train_images'] /= 255
        self.data['test_images'] /= 255

        self.data['train_labels'] = Utility.one_hot_encode(self.data['train_labels'])

        assert self.data['train_labels'].shape == (50000, 10)

In [214]:
class Convolution2D:
    def __init__(self, num_out_channel, filter_size, stride, padding_size):
        self.num_out_channel = num_out_channel
        self.filter_size = filter_size
        self.stride = stride
        self.padding_size = padding_size
        self.h_prev, self.w_prev, self.num_channel_prev = None, None, None
        self.h_new, self.w_new = None, None
        self.W = None
        self.b = None
        self.output_tensor = None
        self.cache = {}
        self.relu_activation = ReLUActivation()

    def initialize_output_dimensions(self, prev_layer_output_dim):
        """
        Initializes output dimensions with the dimension of the previous layers
        :param prev_layer_output_dim: output dimension of the layer immediately before this layer
        :return: None
        """
        self.h_prev, self.w_prev , self.num_channel_prev = prev_layer_output_dim
        self.h_new = (self.h_prev - self.filter_size + 2 * self.padding_size) // self.stride + 1
        self.w_new = (self.w_prev - self.filter_size + 2 * self.padding_size) // self.stride + 1

    def initialize_weights_biases(self):
        """
        Initializes weights with the proper dimensions
        :return:
        """
        self.W = np.random.randn(self.filter_size, self.filter_size, self.num_channel_prev, self.num_out_channel)
        self.b = np.random.randn(1, 1, 1, self.num_out_channel)

    def forward(self, Z_prev, is_training):
        """
        Performs a forward operation of the convolution layer
        :param Z_prev: The activation of the previous layer
        :param is_training: whether we are in training mode or not
        :return:
        """

        assert Z_prev.shape == (self.h_prev, self.w_prev, self.num_channel_prev)
        Z_prev = np.array(Z_prev, copy=True)

        # create zero padded Z_prev
        Z_prev_padded = Utility.zero_pad(Z_prev, self.padding_size)

        # Apply convolution operation over this zero padded previous activation
        for row in range(self.h_new):

            row_start = row * self.stride

            for col in range(self.w_new):

                col_start = col *  self.stride

                for output_channel_index in range(self.num_out_channel):

                    Z_prev_windowed = Z_prev_padded[
                        row_start : row_start + self.filter_size,
                        col_start : col_start + self.filter_size,
                        :
                    ]

                    conv_step_W = self.W[:, :, :, output_channel_index]
                    conv_step_b = self.b[:, :, :, output_channel_index]

                    self.output_tensor[row, col, output_channel_index] = Utility.convolve_single_step(Z_prev_windowed, conv_step_W, conv_step_b)

        # asserting output shape
        assert(self.output_tensor.shape == (self.h_new, self.w_new, self.num_out_channel))

        if is_training:
            # cache some values
            pass

        # perform activation element wise in this case
        self.output_tensor = self.relu_activation.activation_f(self.output_tensor)

    def forward_batch(self, Z_prev, is_training=True):
        """
        Performs a forward operation of the convolution layer
        :param Z_prev: The activation of the previous layer
        :param is_training: whether we are in training mode or not
        :return:
        """
        batch_size = Z_prev.shape[0]
        assert Z_prev.shape == (batch_size, self.h_prev, self.w_prev, self.num_channel_prev)

        self.output_tensor = np.zeros((batch_size, self.h_new, self.w_new, self.num_out_channel))
        Z_prev = np.array(Z_prev, copy=True)

        # create zero padded Z_prev
        print(f'Z prev shape:{Z_prev.shape}')
        Z_prev_padded = Utility.zero_pad(Z_prev, self.padding_size)

        # Apply convolution operation over this zero padded previous activation
        for image_index in range(batch_size):
            current_Z_prev_padded = Z_prev_padded[image_index] # choosing a single tensor from the batch
            for row in range(self.h_new):

                row_start = row * self.stride

                for col in range(self.w_new):

                    col_start = col *  self.stride

                    for output_channel_index in range(self.num_out_channel):

                        Z_prev_windowed = current_Z_prev_padded[
                                        row_start : row_start + self.filter_size,
                                        col_start : col_start + self.filter_size,
                                        :
                                        ]

                        conv_step_W = self.W[:, :, :, output_channel_index]
                        conv_step_b = self.b[:, :, :, output_channel_index]

                        self.output_tensor[image_index, row, col, output_channel_index] = Utility.convolve_single_step(Z_prev_windowed, conv_step_W, conv_step_b)

        # asserting output shape
        assert(self.output_tensor.shape == (batch_size, self.h_new, self.w_new, self.num_out_channel))
        print(self.output_tensor.shape)

        if is_training:
            # cache some values
            pass

        # perform activation element wise in this case
        self.output_tensor = self.relu_activation.activation_f(self.output_tensor)

    def forward_batch_i(self, Z_prev, is_training=True):
        """
        Performs a forward operation of the convolution layer
        :param Z_prev: The activation of the previous layer
        :param is_training: whether we are in training mode or not
        :return:
        """
        batch_size = Z_prev.shape[0]
        assert Z_prev.shape == (batch_size, self.h_prev, self.w_prev, self.num_channel_prev)

        self.output_tensor = np.zeros((batch_size, self.h_new, self.w_new, self.num_out_channel))
        Z_prev = np.array(Z_prev, copy=True)

        # create zero padded Z_prev
        print(f'Z prev shape:{Z_prev.shape}')
        Z_prev_padded = Utility.zero_pad(Z_prev, self.padding_size)

        # Apply convolution operation over this zero padded previous activation
        for row in range(self.h_new):

            row_start = row * self.stride

            for col in range(self.w_new):

                col_start = col *  self.stride

                for output_channel_index in range(self.num_out_channel):

                    Z_prev_windowed = Z_prev_padded[:,row_start : row_start + self.filter_size,
                                            col_start : col_start + self.filter_size,:]

                    conv_step_W = self.W[:, :, :, output_channel_index]
                    conv_step_b = self.b[:, :, :, output_channel_index]

                    # print('z shape', Z_prev_windowed.shape)
                    # print('w shape', conv_step_W.shape)

                    # self.output_tensor[:, row, col, output_channel_index] = np.sum(
                    #     Z_prev_windowed * conv_step_W,
                    #     axis=(1,2,3)
                    # ) + conv_step_b

                    self.output_tensor[:, row, col, output_channel_index] = Utility.convolve_single_step_over_batch(
                        Z_prev_windowed, conv_step_W, conv_step_b
                    )

            # asserting output shape
            assert(self.output_tensor.shape == (batch_size, self.h_new, self.w_new, self.num_out_channel))
            print('output tensor shape:', self.output_tensor.shape)

            if is_training:
                # cache some values
                pass

            # perform activation element wise in this case
            # self.output_tensor = self.relu_activation.activation_f(self.output_tensor)

    def backward(self):
        pass

    def update_CNN_parameters(self, dW : np.array, db: np.array):
        self.W = self.W - dW
        self.b = self.b - db

    def print_layer_dimensions(self):
        print(f'Output Tensor Dimensions: {self.output_tensor.shape}')
        print(f'Weight Dimension: {self.W.shape}')
        print(f'Bias Dimension: {self.b.shape}')

    def get_output_dimension(self) -> Tuple:
        return self.h_new, self.w_new, self.num_out_channel

In [ ]:
class Flatten:
    def __init__(self):
        self.input_dim = None
        self.output_tensor = None
        self.output_dim = None
        self.h_prev, self.w_prev, self.num_channel_prev = None, None, None

    def initialize_flatten_layer_dimensions(self, prev_layer_output_dim):
        """
        :param prev_layer_output_dim: prev layer output of shape (new_h, new_w, new_channel)
        :return: None
        """
        self.h_prev, self.w_prev, self.num_channel_prev  = prev_layer_output_dim
        self.output_dim = self.h_prev * self.w_prev * self.num_channel_prev

    def forward(self, Z_prev: np.array) -> np.array:
        self.output_tensor = Z_prev.rehsape(Z_prev.shape[0], Z_prev.shape[1] * Z_prev.shape[2] * Z_prev.shape[3])

        assert self.output_tensor.shape[1] == self.output_dim

    def get_output_dims(self):
        return self.output_dim

    def backward(self):
        pass

In [50]:
class MaxPool:
    def __init__(self, filter_size, stride):
        self.filter_size = filter_size
        self.stride = stride
        self.h_prev, self.w_prev, self.num_channel_prev = None, None, None
        self.h_new, self.w_new, self.num_out_channel = None, None, None
        self.output_tensor = None
        self.cache = {}

    def initialize_max_pool_params(self, prev_layer_output_dim):
        """
        Initializes output dimensions with the dimension of the previous layers
        :param prev_layer_output_dim: output dimension of the layer immediately before this layer
        :return: None
        """
        self.h_prev, self.w_prev , self.num_channel_prev = prev_layer_output_dim
        self.h_new = int((self.h_prev - self.filter_size) / self.stride + 1)
        self.w_new = int((self.w_prev - self.filter_size) / self.stride + 1)
        self.num_out_channel = self.num_channel_prev
        self.output_tensor = np.zeros((self.h_new, self.w_new, self.num_out_channel))

    def forward(self, Z_prev, is_training):

        assert Z_prev.shape == (self.h_prev, self.w_prev, self.num_channel_prev)
        Z_prev = np.array(Z_prev, copy=True)

        # Apply convolution operation over this zero padded previous activation
        for row in range(self.h_new):

            row_start = row * self.stride

            for col in range(self.w_new):

                col_start = col *  self.stride

                for output_channel_index in range(self.num_out_channel):

                    Z_prev_windowed = Z_prev[
                                      row_start : row_start + self.filter_size,
                                      col_start : col_start + self.filter_size,
                                      output_channel_index
                                      ]

                    self.output_tensor[row, col, output_channel_index] = Utility.get_max_pool_window(Z_prev_windowed)

        assert self.output_tensor.shape == (self.h_new, self.w_new, self.num_out_channel)
        if is_training:
            pass

    def print_layer_dimensions(self):
        print(f'Output Tensor Dimensions: {self.output_tensor.shape}')

    def backward(self):
        pass

    def get_output_dimension(self):
        return self.h_new, self.w_new, self.num_out_channel

In [81]:
class DenseLayer:
    def __init__(self, num_units, activation_obj):
        self.W = None
        self.b = None
        self.dW = None
        self.db = None
        self.num_units = num_units
        self.cache = {}
        self.output_tensor = None
        self.activation_obj = activation_obj

    def initialize_weights_biases(self, prev_flat_layer_output_dim):
        self.W = np.random.randn(self.num_units, prev_flat_layer_output_dim)
        self.b = np.random.randn(self.num_units, 1) # will be broadcast to (hidden_units, batch_size) before addition

    def forward(self, Z_prev):
        """
        :param Z_prev: tensor of shape (1, prev_flattened_shape)
        :return:
        """
        assert Z_prev.shape[1] == self.W.shape[1]

        Z_prev = np.array(Z_prev, copy=True)
        Z_prev = Z_prev.reshape(Z_prev.shape[1], Z_prev.shape[0])
        self.output_tensor = self.activation_obj.activation_f(np.dot(self.W, Z_prev) + self.b)

        # assert the output tensor shape should be (num_hidden_units, batch size)
        assert self.output_tensor.shape == (self.num_units, Z_prev.shape[1])

        # Perform softmax activation upon the elements
        self.output_tensor = self.activation_obj.activation_f(self.output_tensor)


    def backward(self):
        pass

    def get_output_dimension(self):
        return self.output_tensor.shape[0]

    def print_layer_dimensions(self):
        print(f'Output Tensor Dimensions: {self.output_tensor.shape}')
        print(f'Weight Dimension: {self.W.shape}')
        print(f'Bias Dimension: {self.b.shape}')

In [213]:
class Utility:

    @staticmethod
    def one_hot_encode(y_true):
        # Define the One-hot Encoder
        ohe = preprocessing.OneHotEncoder()
        ohe.fit(y_true)
        y_true = ohe.transform(y_true).toarray()
        return y_true

    @staticmethod
    def zero_pad(tensor, pad_size):
        """
        :param tensor: tensor of shape (batch_size, h, w, num_channel)
        :return: padded tensor of shape (h + 2 * pad_size, w + 2 * pad_size, num_channel)
        """
        return np.pad(tensor, ((0,0), (pad_size, pad_size), (pad_size, pad_size), (0,0)), mode='constant', constant_values=0)

    @staticmethod
    def convolve_single_step(Z_prev_windowed, W, b):
        """
        :param Z_prev_windowed: window of shape (F, F, num_channel_Z_prev)
        :param W: kernel/filter/weight of shape (F, F, num_channel_Z_prev)
        :param b: bias term of shape (1, 1, 1)
        :return: scaler convolved value
        """
        return np.multiply(Z_prev_windowed, W).sum() + float(b)

    @staticmethod
    def convolve_single_step_over_batch(tensor, W, b):
        return np.sum(tensor * W, axis=(1,2,3)) + b


    @staticmethod
    def get_max_pool_window(Z_prev_windowed):
        return Z_prev_windowed.max()

    @staticmethod
    def create_mini_batches(X, Y, mini_batch_size):
        total_data = X.shape[0]
        for index in range(0, total_data, mini_batch_size):
            start_index = index
            end_index = min(start_index + mini_batch_size, total_data)
            yield X[start_index: end_index,...], Y[start_index: end_index, ...]

In [53]:
class ReLUActivation:
    @staticmethod
    def activation_f(tensor):
        return np.max(tensor, 0)

    @staticmethod
    def d_relu(tensor):
        return np.where(tensor > 0, 1, 0)

In [ ]:
class SoftmaxActivation:
    @staticmethod
    def activation_f(tensor):
        exponent = np.exp(tensor)
        return tensor/np.sum(exponent)

    @staticmethod
    def d_softmax(tensor):
        pass

In [ ]:
class crossEntropyLoss:
    pass

In [54]:
class GradientDescent:
    pass

In [55]:
class Model:
    def __init__(self):
        pass

    def compile(self):
        pass

    def train(self):
        pass

    def predict(self):
        pass

In [52]:
mnist = MnistDataLoader('./dataset/mnist')
mnist.load_mnist()
mnist.preprocess_data()

In [12]:
mnist.data[mnist.data_list[1]][0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [20]:
a = np.random.randn(3,3,3,5)
a[:,:,:,1].shape

(3, 3, 3)

In [217]:
np.random.seed(1)
A_prev = np.random.randn(10,5,7,4) # h, w, c of previous layer
cnn = Convolution2D(num_out_channel=8, filter_size=3, stride=2, padding_size=1)
cnn.initialize_output_dimensions((A_prev.shape[1], A_prev.shape[2], A_prev.shape[3]))
cnn.initialize_weights_biases()
#cnn.print_layer_dimensions()

In [218]:
cnn.forward_batch_i(A_prev, is_training=True)

Z prev shape:(10, 5, 7, 4)
output tensor shape: (10, 3, 4, 8)
output tensor shape: (10, 3, 4, 8)
output tensor shape: (10, 3, 4, 8)


In [219]:
print(cnn.output_tensor.shape)
print("Z's mean =\n", np.mean(cnn.output_tensor))
print("Z[3,2,1] =\n", cnn.output_tensor[3,2,1])

(10, 3, 4, 8)
Z's mean =
 0.6923608807576933
Z[3,2,1] =
 [-1.28912231  2.27650251  6.61941931  0.95527176  8.25132576  2.31329639
 13.00689405  2.34576051]


In [204]:
# Case 1: stride of 1
np.random.seed(1)
A_prev = np.random.randn(5, 5, 3)
maxpool = MaxPool(filter_size=3, stride=1)
maxpool.initialize_max_pool_params(A_prev.shape)
maxpool.forward(A_prev, True)
maxpool.print_layer_dimensions()
print(maxpool.output_tensor)

3
3
3
Output Tensor Dimensions: (3, 3, 3)
[[[1.74481176 0.90159072 1.65980218]
  [1.74481176 1.46210794 1.65980218]
  [1.74481176 1.6924546  1.65980218]]

 [[1.14472371 0.90159072 2.10025514]
  [1.14472371 0.90159072 1.65980218]
  [1.14472371 1.6924546  1.65980218]]

 [[1.13162939 1.51981682 2.18557541]
  [1.13162939 1.51981682 2.18557541]
  [1.13162939 1.6924546  2.18557541]]]


In [ ]:

hparameters = {"stride" : 1, "f": 3}

A, cache = pool_forward(A_prev, hparameters)
print("mode = max")
print("A.shape = " + str(A.shape))
print("A =\n", A)
print()
A, cache = pool_forward(A_prev, hparameters, mode = "average")
print("mode = average")
print("A.shape = " + str(A.shape))
print("A =\n", A)

In [60]:
a = np.array([1,23])
b = np.array(a, copy=True)
b == a

array([ True,  True])

In [56]:
cifer_dataloader = Cifer10DataLoader('/home/akil/Work/Work/Academics/4-2/ML/Assignment-3/dataset/cifer-10/cifar-10-python/cifar-10-batches-py')
cifer_dataloader.concatenate_data()
cifer_dataloader.preprocess_data()

(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
(10000,)
Shape: (50000, 32, 32, 3)
Shape: (50000, 1)
Shape: (10000, 32, 32, 3)
Shape: (10000, 1)


In [57]:
cifer_dataloader.data['train_labels'][0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [89]:
cifer_dataloader.data['train_images'].shape

(50000, 32, 32, 3)

In [90]:
cifer_dataloader.data['train_labels'].shape

(50000, 10)

In [93]:
total_data = cifer_dataloader.data['train_images'].shape[0]
mini_batch_size = 32
p = np.random.permutation(total_data)
cifer_dataloader.data['train_images'], cifer_dataloader.data['train_labels'] = cifer_dataloader.data['train_images'][p, :], cifer_dataloader.data['train_labels'][p, :]

In [141]:
X = cifer_dataloader.data['train_images']
Y = cifer_dataloader.data['train_labels']

In [142]:
for X, Y in Utility.create_mini_batches(X, Y, mini_batch_size=64):
    print(X.shape)
    print(Y.shape)

(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
(64, 32, 32, 3)
(64, 10)
